In [126]:
import torch
import torch.nn.functional as F
import random

In [2]:
words = open('names.txt', 'r').read().splitlines()

In [3]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
# chars.insert(0, '.')
len(chars)

26

In [25]:
dchars = []
for c in chars:
    dchars.append(f'.{c}')
for c in chars:
    for c2 in chars:
        dchars.append(f'{c}{c2}')

two_char_variations = len(dchars)
for c in chars:
    dchars.append(c)
dchars.append('.')
print(f"dchars len: {len(dchars)}")
stoi = {s:i for i,s in enumerate(dchars)}
itos = {i:s for s,i in stoi.items()}

len(stoi)

dchars len: 729


729

In [62]:
len(dchars)

729

In [7]:
N = torch.zeros((two_char_variations, 27), dtype=torch.int32)
N.shape

torch.Size([702, 27])

In [8]:
g = torch.Generator()


In [30]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for i, c in enumerate(chs[:-2]):
        str1 = ''.join(chs[i:i+2])
        str2 = chs[i+2]
        ix1 = stoi[str1]
        ix2 = stoi[str2]
        xs.append(ix1)
        ys.append(ix2)
#         print(str1, str2, ix1, ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

number of examples:  196113


In [33]:
ys.shape

torch.Size([196113])

In [43]:
g = torch.Generator()
W = torch.randn((729, 729), generator=g, requires_grad=True)

In [193]:
for k in range(10):
    #forward pass
    xenc = F.one_hot(xs, num_classes=729).float()
    logits = xenc @ W #predicct log counts
    counts = logits.exp() #counts
    probs = counts / counts.sum(1, keepdims=True) #probabilities for the next char
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())
    #backward pass
    W.grad = None
    loss.backward()
    
    W.data += -200 * W.grad

2.4067416191101074
2.4056434631347656
2.404552459716797
2.403468608856201
2.4023923873901367
2.401322841644287
2.4002604484558105
2.399205207824707
2.3981568813323975
2.397115468978882


In [207]:
for i in range(10):
    out = []
    ix = random.randint(0, 28)
    out.append(itos[ix])
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=729).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 728: #single dot index
            break
        last_2_chars = (''.join(out))[-2:]
        ix = stoi[last_2_chars]
    print(''.join(out)[1:-1])
            

rakg.parlon
nee
jogh
re
fattuban
ni
vivexgoe
cola
even
olain
